>>> <hr> 모델 실습 <hr>

- SentenceClassifier 클래스는 임베딩 층을 구성할 떄 사용하는 단어 사전 크기(n_vocab)와 순환 신경망 클래스와 장단기 메모리 클래스에서 사용하는 매개변수를 입력으로 전달받는다.
- 모델 종류(model_type) 매개변수로 순환 신경망을 사용할지, 장단기 메모리를 사용할지 설정한다.

- 초기화 메서드에서는 SentenceClassifier 클래스에 입력된 함수의 매개변수에 따라 모델 구조를 미세 조정한다.
- 분류기 계층은 모델을 양방향으로 구성한다면 전달되는 입력 채널 수가 달라지므로 분류기 계층을 현재 모델 구조에 맞게 변경한다.

- 순방향 메서드에서는 입력받은 정수 인코딩을 임베딩 계층에 통과시켜 임베딩 값을 얻는다. 그리고 얻은 임베딩 값을 모델에 입ㅇ력하여 출력값을 얻는다.
- 출력값(output)의 마지막 시점만 활용할 예정이므로 [:, -1, :]으로 마지막 시점의 결괏값만 분리해 분류기 계층에 전달한다.

In [8]:
# 문장 분류 모델
from torch import nn

class SentenceClassifier(nn.Module):
    def __init__(
            self,
            n_vocab,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout=0.5,
            bidirectional=True,
            model_type="lstm"
    ):
        super().__init__()

        self.embedding=nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        if model_type =="rnn":
            self.model=nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type =="lstm":
            self.model=nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        if bidirectional:
            self.classifier=nn.Linear(hidden_dim * 2,1)
        else:
            self.classifier=nn.Linear(hidden_dim, 1)
        self.dropout=nn.Dropout(dropout)
    
    def forward(self, inputs):
        embeddings=self.embedding(inputs)
        output, _=self.model(embeddings)
        last_output=output[:, -1, :]
        last_output=self.dropout(last_output)
        logits=self.classifier(last_output)
        return logits

- SentenceClassifier 클래스를 선언했다면 모델 학습에 사용할 데이터세트를 불러온다. 데이터세트는 코포라 라이브러리의 네이버 영화 리뷰 감정 분석 데이터세트를 불러온다.

In [9]:
# 데이터세트 불러오기
import pandas as pd
from Korpora import Korpora
#%pip install tabulate

corpus=Korpora.load("nsmc")
corpus_df=pd.DataFrame(corpus.test)

train_data=corpus_df.sample(frac=0.9, random_state=42)
test_data=corpus_df.drop(train_data.index)

print(train_data.head(5).to_markdown())
print("Training Data Size :", len(train_data))
print("Testing Data Size :", len(test_data))


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\LG\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\LG\Kor

In [10]:
# 데이터 토큰화 및 단어 사전 구축
from konlpy.tag import Okt
from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens):
    counter=Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab=special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

tokenizer=Okt()
train_tokens=[tokenizer.morphs(review) for review in train_data.text]
test_tokens=[tokenizer.morphs(review) for review in test_data.text]

vocab=build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id={token: idx for idx, token in enumerate(vocab)}
id_to_token={idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '.', '이', '영화', '의', '..', '가', '에', '...']
5002


In [11]:
# 정수 인코딩 및 패딩
import numpy as np

def pad_sequences(sequences, max_length, pad_value):
    result=list()
    for sequence in sequences:
        sequence=sequence[:max_length]
        pad_length=max_length-len(sequence)
        padded_sequence=sequence+[pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

unk_id=token_to_id['<unk>']
train_ids=[
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids=[
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length=32
pad_id=token_to_id["<pad>"]
train_ids=pad_sequences(train_ids, max_length, pad_id)
test_ids=pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[ 223 1716   10 4036 2095  193  755    4    2 2330 1031  220   26   13
 4839    1    1    1    2    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[3307    5 1997  456    8    1 1013 3906    5    1    1   13  223   51
    3    1 4684    6    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [12]:
# 데이터로더 적용
import torch
from torch.utils.data import TensorDataset, DataLoader

train_ids=torch.tensor(train_ids)
test_ids=torch.tensor(test_ids)

train_labels=torch.tensor(train_data.label.values, dtype=torch.float32)
test_labels=torch.tensor(test_data.label.values, dtype=torch.float32)

train_dataset=TensorDataset(train_ids, train_labels)
test_dataset=TensorDataset(test_ids, test_labels)

train_loader=DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=16, shuffle=False)

In [13]:
# 손실 함수와 최적화 함수 정의
from torch import optim

n_vocab=len(token_to_id)
hidden_dim=64
embedding_dim=128
n_layers=2

device="cuda" if torch.cuda.is_available() else "cpu"
classifier=SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)
criterion=nn.BCEWithLogitsLoss().to(device)
optimizer=optim.RMSprop(classifier.parameters(), lr=0.001)

In [14]:
# 모델 학습 및 테스트
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses=list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids=input_ids.to(device)
        labels=labels.to(device).unsqueeze(1)

        logits=model(input_ids)
        loss=criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval==0:
            print(f"Train Loss {step} : {np.mean(losses)}")

def test(model, datasets, criterion, device):
    model.eval()
    losses=list()
    corrects=list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids=input_ids.to(device)
        labels=labels.to(device).unsqueeze(1)

        logits=model(input_ids)
        loss=criterion(logits, labels)
        losses.append(loss.item())
        yhat=torch.sigmoid(logits)>.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f'Val Loss: {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}')

epochs=5
interval=500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6843744516372681
Train Loss 500 : 0.6935779821135089
Train Loss 1000 : 0.688331649853633
Train Loss 1500 : 0.6736074270684269
Train Loss 2000 : 0.6653966238384305
Train Loss 2500 : 0.6562119460687404
Val Loss: 0.6003798047384135, Val Accuracy : 0.6968
Train Loss 0 : 0.417904794216156
Train Loss 500 : 0.5810391278918869
Train Loss 1000 : 0.5635296040838891
Train Loss 1500 : 0.5472045288552132
Train Loss 2000 : 0.536578912062683
Train Loss 2500 : 0.521932823986399
Val Loss: 0.46953022532379285, Val Accuracy : 0.7814
Train Loss 0 : 0.35736462473869324
Train Loss 500 : 0.4305190174225086
Train Loss 1000 : 0.4259616179051218
Train Loss 1500 : 0.4241200108267243
Train Loss 2000 : 0.4193279863535792
Train Loss 2500 : 0.4182979257636574
Val Loss: 0.42167099158223065, Val Accuracy : 0.7986
Train Loss 0 : 0.23584696650505066
Train Loss 500 : 0.36406485047823417
Train Loss 1000 : 0.3681068767334793
Train Loss 1500 : 0.3697245377131417
Train Loss 2000 : 0.3688083860217065
Train Lo

In [15]:
# 학습된 모델로부터 임베딩 추출
token_to_embedding=dict()
embedding_matrix=classifier.embedding.weight.detach().cpu().numpy()

for word, emb in zip(vocab, embedding_matrix):
    token_to_embedding[word]=emb

token=vocab[1000]
print(token, token_to_embedding[token])

보고싶다 [-0.40999418  1.6593337   1.1083715  -0.24211055  1.283378    1.310663
  0.15313952 -0.42823     1.845338   -1.2713537  -2.4918156   0.5678896
  2.0654538   0.350038    1.2830559   1.0796698  -1.4274625   0.3354824
 -0.231855   -0.9960354  -0.68714935  0.5737285   2.658381   -1.4902339
 -1.0307599  -0.7208531  -1.2912513  -0.7472857  -0.3688705   0.39077032
  0.19352224 -1.2516541  -0.7396949   0.72476286 -1.312093    1.2689428
  0.6009034  -1.0034074   0.11286522 -1.7167953  -0.81556237 -0.3037526
  0.48595783  1.0280796   0.75985837 -0.27178425  0.65489763 -1.0963597
  0.8265645  -0.23160197  3.617519    0.7064647   2.022217   -0.9964674
 -0.22540146  0.6039761   0.81769395  0.338092    0.6824848   0.91780096
 -0.01283852  0.8933017  -1.8751456  -0.9103439   0.51728314  1.4783889
 -3.0495174   1.7487664   0.38325652  0.7961067   1.1604681  -0.13437289
 -0.4388663  -0.562142   -0.5991527   0.43821457  0.2992261  -0.43645743
  0.6861941   2.4266891   0.7141187  -0.24047372 -1.8588

In [16]:
#%pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------- ----------------------------- 6.0/24.0 MB 33.5 MB/s eta 0:00:01
   ----------------------- ---------------- 13.9/24.0 MB 36.4 MB/s eta 0:00:01
   -------------------------- ------------- 16.0/24.0 MB 30.5 MB/s eta 0:00:01
   ------------------------------------ --- 21.8/24.0 MB 27.0 MB/s eta 0:00:01
   ---------------------------------------- 24.0/24.0 MB 26.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
